# Neural Networks

Source: https://pytorch.org/tutorials/beginner/blitz/neural_networks_tutorial.html#sphx-glr-beginner-blitz-neural-networks-tutorial-py

Example about cassifies digit images:

<img src="https://pytorch.org/tutorials/_images/mnist.png">

A typical training procedure for a neural network is as follows:

- Define the neural network that has some learnable parameters (or weights)
- Iterate over a dataset of inputs
- Process input through the network
- Compute the loss (how far is the output from being correct)
- Propagate gradients back into the network’s parameters
- Update the weights of the network, typically using a simple update rule: `weight` = `weight` - `learning_rate` * `gradient`


## Define the network

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        # 1 input image channel, 6 output channels, 3x3 square convolution
        # kernel
        self.conv1 = nn.Conv2d(1, 6, 3)
        self.conv2 = nn.Conv2d(6, 16, 3)
        # an affine operation: y = Wx + b
        self.fc1 = nn.Linear(16 * 6 * 6, 120)  # 6*6 from image dimension
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        # Max pooling over a (2, 2) window
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        # If the size is a square you can only specify a single number
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

    def num_flat_features(self, x):
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features


net = Net()
print(net)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=576, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


In [3]:
params = list(net.parameters())
print(len(params))
print(params[0].size()) #conv1's.weight

10
torch.Size([6, 1, 3, 3])


In [4]:
input = torch.randn(1, 1, 32, 32)
print(input)
out = net(input)
print(out)

tensor([[[[-2.1950, -1.2435,  1.1125,  ..., -0.6578, -0.3214, -1.5464],
          [ 0.0401, -0.1778,  1.3368,  ...,  0.3988,  0.9840,  0.9993],
          [ 0.2486, -1.5186,  0.2485,  ..., -0.1903,  0.4236,  0.7167],
          ...,
          [-1.3805, -0.2879, -0.5170,  ..., -0.5685, -0.8285, -0.8597],
          [ 0.2067,  0.1770,  0.9008,  ..., -0.3814,  0.5116, -0.4198],
          [-0.3054, -0.6854,  0.7084,  ..., -0.9960,  0.6896,  1.7246]]]])
tensor([[ 0.0668, -0.0323,  0.0379,  0.1274, -0.1228, -0.0773, -0.1631,  0.0277,
         -0.0355, -0.0014]], grad_fn=<AddmmBackward>)


In [5]:
net.zero_grad()
out.backward(torch.randn(1,10))